<a href="https://colab.research.google.com/github/yaswanthlakkasani2901/Financeial-Management/blob/main/FinanceSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3

def initialize_db():
    try:
        db = sqlite3.connect("finance.db")
        cursor = db.cursor()


        cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL
        )
        """)

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS income (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            amount REAL NOT NULL,
            date TEXT NOT NULL,
            description TEXT
        )
        """)

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS expenses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            amount REAL NOT NULL,
            date TEXT NOT NULL,
            description TEXT
        )
        """)

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS budgets (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            category TEXT NOT NULL,
            amount REAL NOT NULL
        )
        """)

        db.commit()
    except sqlite3.Error as e:
        print(f"An error occurred while initializing the database: {e}")
    finally:
        db.close()

initialize_db()

In [ ]:
import getpass
import sqlite3

class User:
    def __init__(self, username, password):
        self.username = username
        self.password = password

def register_user():
    try:
        username = input("Enter a username: ")
        password = getpass.getpass("Enter a password: ")

        db = sqlite3.connect("finance.db")
        cursor = db.cursor()
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
        db.commit()
        print("Registration successful!")
    except sqlite3.Error as e:
        print(f"An error occurred during registration: {e}")
    finally:
        db.close()

def login_user():
    try:
        username = input("Enter your username: ")
        password = getpass.getpass("Enter your password: ")

        db = sqlite3.connect("finance.db")
        cursor = db.cursor()
        cursor.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, password))
        user = cursor.fetchone()

        if user:
            print("Login successful!")
            return True
        else:
            print("Invalid username or password.")
            return False
    except sqlite3.Error as e:
        print(f"An error occurred during login: {e}")
        return False
    finally:
        db.close()


In [ ]:
class Budget:
    def __init__(self, category, amount):
        self.category = category
        self.amount = amount

def set_budget():
    try:
        category = input("Enter budget category: ")
        amount = float(input("Enter budget amount: "))
        db = sqlite3.connect("finance.db")
        cursor = db.cursor()
        cursor.execute("INSERT INTO budgets (category, amount) VALUES (?, ?)", (category, amount))
        db.commit()
        print(f"Budget set for {category}: ${amount}")
    except sqlite3.Error as e:
        print(f"An error occurred while setting the budget: {e}")
    finally:
        db.close()

def check_budget():
    category = input("Enter budget category: ")

    try:
        db = sqlite3.connect("finance.db")
        cursor = db.cursor()


        cursor.execute("SELECT amount FROM budgets WHERE category = ?", (category,))
        budget_amount = cursor.fetchone()
        if budget_amount is None:
            print(f"No budget found for category '{category}'")
            return

        budget_amount = budget_amount[0]


        cursor.execute("SELECT SUM(amount) FROM expenses WHERE description = ?", (category,))
        total_expenses = cursor.fetchone()[0]
        total_expenses = total_expenses if total_expenses is not None else 0.0

        if total_expenses > budget_amount:
            print(f"You have exceeded your budget for {category}. Budget: ${budget_amount}, Expenses: ${total_expenses}")
        else:
            print(f"You're within budget for {category}. Budget: ${budget_amount}, Expenses: ${total_expenses}")

    except sqlite3.Error as e:
        print(f"An error occurred while checking the budget: {e}")
    finally:
        db.close()


In [ ]:
class Income:
    def __init__(self, amount, date, description):
        self.amount = amount
        self.date = date
        self.description = description

class Expense:
    def __init__(self, amount, date, description):
        self.amount = amount
        self.date = date
        self.description = description

def add_income():
    try:
        amount = float(input("Enter income amount: "))
        date = input("Enter income date (YYYY-MM-DD): ")
        description = input("Enter income description: ")

        db = sqlite3.connect("finance.db")
        cursor = db.cursor()
        cursor.execute("INSERT INTO income (amount, date, description) VALUES (?, ?, ?)", (amount, date, description))
        db.commit()
        print("Income added successfully.")
    except sqlite3.Error as e:
        print(f"An error occurred while adding income: {e}")
    finally:
        db.close()

def add_expense():
    try:
        amount = float(input("Enter expense amount: "))
        date = input("Enter expense date (YYYY-MM-DD): ")
        description = input("Enter expense description: ")

        db = sqlite3.connect("finance.db")
        cursor = db.cursor()
        cursor.execute("INSERT INTO expenses (amount, date, description) VALUES (?, ?, ?)", (amount, date, description))
        db.commit()
        print("Expense added successfully.")
    except sqlite3.Error as e:
        print(f"An error occurred while adding expense: {e}")
    finally:
        db.close()


In [ ]:
import datetime
import calendar

class FinancialReport:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date

    def calculate_total_income(self):
        try:
            db = sqlite3.connect("finance.db")
            cursor = db.cursor()
            cursor.execute("SELECT SUM(amount) FROM income WHERE date BETWEEN ? AND ?", (self.start_date, self.end_date))
            total_income = cursor.fetchone()[0]
            return total_income if total_income else 0.0
        except sqlite3.Error as e:
            print(f"An error occurred while calculating income: {e}")
            return 0.0
        finally:
            db.close()

    def calculate_total_expenses(self):
        try:
            db = sqlite3.connect("finance.db")
            cursor = db.cursor()
            cursor.execute("SELECT SUM(amount) FROM expenses WHERE date BETWEEN ? AND ?", (self.start_date, self.end_date))
            total_expenses = cursor.fetchone()[0]
            return total_expenses if total_expenses else 0.0
        except sqlite3.Error as e:
            print(f"An error occurred while calculating expenses: {e}")
            return 0.0
        finally:
            db.close()

    def calculate_savings(self):
        total_income = self.calculate_total_income()
        total_expenses = self.calculate_total_expenses()
        return total_income - total_expenses

def generate_monthly_report():
    current_date = datetime.date.today()
    start_date = current_date.replace(day=1)
    _, last_day = calendar.monthrange(current_date.year, current_date.month)
    end_date = current_date.replace(day=last_day)

    report = FinancialReport(start_date, end_date)
    print("\nMonthly Report:")
    print("Total Income: $", report.calculate_total_income())
    print("Total Expenses: $", report.calculate_total_expenses())
    print("Savings: $", report.calculate_savings())

def generate_yearly_report():
    current_date = datetime.date.today()
    start_date = current_date.replace(month=1, day=1)
    end_date = current_date.replace(month=12, day=31)

    report = FinancialReport(start_date, end_date)
    print("\nYearly Report:")
    print("Total Income: $", report.calculate_total_income())
    print("Total Expenses: $", report.calculate_total_expenses())
    print("Savings: $", report.calculate_savings())


In [ ]:
def main():
    while True:
        print("\nFinancial Management System")
        print("1. Register")
        print("2. Login")
        print("3. Add Income")
        print("4. Add Expense")
        print("5. Set Budget")
        print("6. Check Budget")
        print("7. Generate Monthly Report")
        print("8. Generate Yearly Report")
        print("9. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            register_user()
        elif choice == '2':
            if login_user():
                continue
        elif choice == '3':
            add_income()
        elif choice == '4':
            add_expense()
        elif choice == '5':
            set_budget()
        elif choice == '6':
            check_budget()
        elif choice == '7':
            generate_monthly_report()
        elif choice == '8':
            generate_yearly_report()
        elif choice == '9':
            print("Goodbye!")
            break
        else:
            print("Invalid choice, please try again.")

if __name__ == "__main__":
    main()


Financial Management System
1. Register
2. Login
3. Add Income
4. Add Expense
5. Set Budget
6. Check Budget
7. Generate Monthly Report
8. Generate Yearly Report
9. Exit
Enter your choice: Hsygsh
Invalid choice, please try again.

Financial Management System
1. Register
2. Login
3. Add Income
4. Add Expense
5. Set Budget
6. Check Budget
7. Generate Monthly Report
8. Generate Yearly Report
9. Exit
Enter your choice: Register
Invalid choice, please try again.

Financial Management System
1. Register
2. Login
3. Add Income
4. Add Expense
5. Set Budget
6. Check Budget
7. Generate Monthly Report
8. Generate Yearly Report
9. Exit
